# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
%run create_tables.py

In [2]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [3]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [4]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [5]:
song_files = get_files('data/song_data')
len(song_files)

71

In [6]:
filepath = song_files[0]
filepath

'/workspace/home/data/song_data/A/B/B/TRABBNP128F932546F.json'

In [7]:
song_df = pd.read_json(filepath, lines = True)
song_df.head()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,AR62SOJ1187FB47BB5,NaN,,NaN,Chase & Status,337.68444,1,SOGVQGJ12AB017F169,Ten Tonne,2005


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [8]:
song_data = (song_df[['song_id', 'title', 'artist_id', 'year', 'duration']].values[0]).tolist()
song_data

['SOGVQGJ12AB017F169', 'Ten Tonne', 'AR62SOJ1187FB47BB5', 2005, 337.68444]

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [9]:
cur.execute(song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [10]:
artist_data = (song_df[['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']].values[0]).tolist()
artist_data

['AR62SOJ1187FB47BB5', 'Chase & Status', '', nan, nan]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [11]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [12]:
log_files = get_files('data/log_data')

In [13]:
filepath = log_files[0]
filepath

'/workspace/home/data/log_data/2018/11/2018-11-04-events.json'

In [14]:
log_df = pd.read_json(filepath, lines = True)
log_df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,None,Logged In,Theodore,M,0,Smith,NaN,free,"Houston-The Woodlands-Sugar Land, TX",GET,Home,1.540306e+12,154,None,200,1541290555796,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,52
1,Professor Longhair,Logged In,Ann,F,0,Banks,214.20363,free,"Salt Lake City, UT",PUT,NextSong,1.540896e+12,124,Mean Ol'World,200,1541292603796,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...,99
2,None,Logged In,Jahiem,M,0,Miles,NaN,free,"San Antonio-New Braunfels, TX",GET,Home,1.540817e+12,42,None,200,1541299033796,"""Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537....",43
3,Gary Hobbs,Logged In,Jahiem,M,1,Miles,245.52444,free,"San Antonio-New Braunfels, TX",PUT,NextSong,1.540817e+12,42,En Mi Mundo,200,1541300092796,"""Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537....",43
4,Lifehouse,Logged In,Jahiem,M,2,Miles,203.59791,free,"San Antonio-New Braunfels, TX",PUT,NextSong,1.540817e+12,42,We'll Never Know,200,1541300337796,"""Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537....",43


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [15]:
log_df = log_df[log_df['page'] == 'NextSong']
log_df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
1,Professor Longhair,Logged In,Ann,F,0,Banks,214.20363,free,"Salt Lake City, UT",PUT,NextSong,1.540896e+12,124,Mean Ol'World,200,1541292603796,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...,99
3,Gary Hobbs,Logged In,Jahiem,M,1,Miles,245.52444,free,"San Antonio-New Braunfels, TX",PUT,NextSong,1.540817e+12,42,En Mi Mundo,200,1541300092796,"""Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537....",43
4,Lifehouse,Logged In,Jahiem,M,2,Miles,203.59791,free,"San Antonio-New Braunfels, TX",PUT,NextSong,1.540817e+12,42,We'll Never Know,200,1541300337796,"""Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537....",43
5,Olivia Ruiz,Logged In,Jahiem,M,3,Miles,254.74567,free,"San Antonio-New Braunfels, TX",PUT,NextSong,1.540817e+12,42,Cabaret Blanco,200,1541300540796,"""Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537....",43
7,Jordan Rudess,Logged In,Cecilia,F,1,Owens,1367.84934,free,"Atlanta-Sandy Springs-Roswell, GA",PUT,NextSong,1.541032e+12,225,Tarkus,200,1541306152796,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:32.0) G...,6


In [16]:
t = pd.to_datetime(log_df['ts'], unit = 'ms')
t.head()

1   2018-11-04 00:50:03.796
3   2018-11-04 02:54:52.796
4   2018-11-04 02:58:57.796
5   2018-11-04 03:02:20.796
7   2018-11-04 04:35:52.796
Name: ts, dtype: datetime64[ns]

In [17]:
time_data = [t.dt.time, t.dt.hour, t.dt.day, t.dt.week, t.dt.month, t.dt.year, t.dt.weekday]

column_labels = ('timestamp', 'hour', 'day', 'week', 'month', 'year', 'weekday')

In [18]:
time_dict = {}
for c in range(len(column_labels)):
    time_dict[column_labels[c]] = time_data[c]
time_df = pd.DataFrame(time_dict)

time_df.tail()

,timestamp,hour,day,week,month,year,weekday
180,21:01:44.796000,21,4,44,11,2018,6
181,22:56:24.796000,22,4,44,11,2018,6
182,22:58:36.796000,22,4,44,11,2018,6
183,23:02:14.796000,23,4,44,11,2018,6
188,23:45:13.796000,23,4,44,11,2018,6


In [19]:
print(time_df.iloc[91,0])

14:02:17.796000


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [20]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [21]:
user_df = log_df[['userId', 'firstName', 'lastName', 'gender', 'level']]


In [22]:
user_df.head()

,userId,firstName,lastName,gender,level
1,99,Ann,Banks,F,free
3,43,Jahiem,Miles,M,free
4,43,Jahiem,Miles,M,free
5,43,Jahiem,Miles,M,free
7,6,Cecilia,Owens,F,free


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [23]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [24]:
#testing the song_select query for 1 line
cur.execute(song_select,(log_df.song[1], log_df.artist[1], log_df.length[1]))
results = cur.fetchone()
print(results)
print(cur.mogrify(song_select, (log_df.song[1], log_df.artist[1], log_df.length[1])))

None
b"SELECT songs.song_id, songs.artist_id FROM songs JOIN artists ON songs.artist_id = artists.artist_id WHERE songs.title = 'Mean Ol''World' AND artists.name = 'Professor Longhair' AND songs.duration = 214.20363"


In [25]:
#Output shows None, None values from the execute statement!!

for index, row in log_df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    
    #troubleshooting printing
    print(row.song, '|', row.artist, '|', row.length)
    print(type(row.song), type(row.artist), type(row.length), '\n')

        
    results = cur.fetchone()   
    if index < 5:
        print (results)
        print(cur.mogrify(song_select, (row.song, row.artist, row.length)))
    
    if results:
        songid, artistid= results
    else:
        songid, artistid = None, None
    
    
    #insert songplay record
    songplay_data = (time_df.timestamp[index], row.userId, row.level, songid, artistid, row.sessionId, row.location, row.userAgent)
   
    
    cur.execute(songplay_table_insert, songplay_data)

    #print statement inserted while troubleshooting
    if index < 5:
        print('\n',row.song, songplay_data, '\n')
    conn.commit()

Mean Ol'World | Professor Longhair | 214.20363
<class 'str'> <class 'str'> <class 'float'> 

None
b"SELECT songs.song_id, songs.artist_id FROM songs JOIN artists ON songs.artist_id = artists.artist_id WHERE songs.title = 'Mean Ol''World' AND artists.name = 'Professor Longhair' AND songs.duration = 214.20363"

 Mean Ol'World (datetime.time(0, 50, 3, 796000), '99', 'free', None, None, 124, 'Salt Lake City, UT', 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; rv:31.0) Gecko/20100101 Firefox/31.0') 

En Mi Mundo | Gary Hobbs | 245.52444
<class 'str'> <class 'str'> <class 'float'> 

None
b"SELECT songs.song_id, songs.artist_id FROM songs JOIN artists ON songs.artist_id = artists.artist_id WHERE songs.title = 'En Mi Mundo' AND artists.name = 'Gary Hobbs' AND songs.duration = 245.52444"

 En Mi Mundo (datetime.time(2, 54, 52, 796000), '43', 'free', None, None, 42, 'San Antonio-New Braunfels, TX', '"Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/5

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [26]:
#conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.

In [27]:
%run etl.py
conn.commit()

71 files found in data/song_data
1/71 files processed.
2/71 files processed.
3/71 files processed.
4/71 files processed.
5/71 files processed.
6/71 files processed.
7/71 files processed.
8/71 files processed.
9/71 files processed.
10/71 files processed.
11/71 files processed.
12/71 files processed.
13/71 files processed.
14/71 files processed.
15/71 files processed.
16/71 files processed.
17/71 files processed.
18/71 files processed.
19/71 files processed.
20/71 files processed.
21/71 files processed.
22/71 files processed.
23/71 files processed.
24/71 files processed.
25/71 files processed.
26/71 files processed.
27/71 files processed.
28/71 files processed.
29/71 files processed.
30/71 files processed.
31/71 files processed.
32/71 files processed.
33/71 files processed.
34/71 files processed.
35/71 files processed.
36/71 files processed.
37/71 files processed.
38/71 files processed.
39/71 files processed.
40/71 files processed.
41/71 files processed.
42/71 files processed.
43/71 file

In [28]:
#testing the sql query WITHOUT songs.title filter
sql_test = """SELECT songs.song_id, songs.artist_id FROM songs LEFT JOIN artists 
ON songs.artist_id = artists.artist_id WHERE artists.name = 'Chase & Status' AND songs.duration = 337.68444"""
cur.execute(sql_test)
results1 = cur.fetchone()
print(results1)


('SOGVQGJ12AB017F169', 'AR62SOJ1187FB47BB5')


In [29]:
#testing the sql query WITH songs.title filter
sql_test_with_title = """SELECT songs.song_id, songs.artist_id FROM songs JOIN artists
ON songs.artist_id = artists.artist_id WHERE songs.title = 'Ten Tonne' AND artists.name = 'Chase & Status' AND songs.duration = 337.68444"""
cur.execute(sql_test_with_title)
results2 = cur.fetchone()
print(results2)

('SOGVQGJ12AB017F169', 'AR62SOJ1187FB47BB5')


In [30]:
#testing the sql query WITH ONLY songs.title filter
sql_test_with_only_title = """SELECT songs.song_id, songs.artist_id FROM songs JOIN artists
ON songs.artist_id = artists.artist_id WHERE songs.title = 'Ten Tonne'"""
cur.execute(sql_test_with_only_title)
results3 = cur.fetchone()
print(results3)

('SOGVQGJ12AB017F169', 'AR62SOJ1187FB47BB5')


In [32]:
#created a list to put the outputs to isolate it from the main database

song_artist_ids = []
i = 0


for index, row in log_df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select2, (row.song,))
    results = cur.fetchone()
    
    if results:
        artistid= results
        songid = None
    else:
        songid, artistid = None, None
    
    #insert songplay record
    song_artist_ids.append([time_df.timestamp[index], row.userId, row.level, songid, artistid, row.sessionId, row.location, row.userAgent])
    cur.execute(songplay_table_insert, song_artist_ids[i])
    i+=1
    
    #print statement inserted while troubleshooting
    if index < 5:
        print(results)
        print(row.song, song_artist_ids[i-1], '\n')
    conn.commit()

None
Mean Ol'World [datetime.time(0, 50, 3, 796000), '99', 'free', None, None, 124, 'Salt Lake City, UT', 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; rv:31.0) Gecko/20100101 Firefox/31.0'] 

None
En Mi Mundo [datetime.time(2, 54, 52, 796000), '43', 'free', None, None, 42, 'San Antonio-New Braunfels, TX', '"Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"'] 

None
We'll Never Know [datetime.time(2, 58, 57, 796000), '43', 'free', None, None, 42, 'San Antonio-New Braunfels, TX', '"Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"'] 

